In [1]:
import ROOT
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib
import sys
from datetime import datetime, timedelta
import numpy as np
class fsm:
    NS = 3
    states={-1:"idle",0:"waiting start",1:"waiting stop",2:"interval ready"}
    stime = timedelta(0)
    etime = timedelta(0)
    dt = timedelta(0)
    curr_st = 0
    dv = False
    time0 =  datetime.strptime("05/25/2022 10:00:00 AM","%m/%d/%Y %I:%M:%S %p")
    TFRMT="%m/%d/%Y %I:%M:%S %p"
    def __init__(self):
        self.curr_st = 0
    def print_state(self):
        print(self.states[curr_st])
    def next_state(self):
        self.curr_st=(self.curr_st+1)%self.NS
    def reset(self):
        self.curr_state = 0
        self.dv=False
    def process_line(self,line):
        if ("all devices online" in line ) and (self.curr_st==-1):
            self.next_state()
        elif ("event stored" in line ) and (self.curr_st==0):
            self.stime = datetime.strptime((reduce(lambda x,y: x +" "+y,line.split()[:3])),self.TFRMT)
            self.next_state()
        elif ("event stored" in line ) and (self.curr_st==1): 
            self.etime = datetime.strptime((reduce(lambda x,y: x +" "+y,line.split()[:3])),self.TFRMT)
            self.next_state()
        elif ("event stored" in line ) and (self.curr_st!=0) and (self.curr_st!=1): 
            self.reset()            
            self.next_state()
        elif ("sending power cycle" in line ) and (self.curr_st==2):
            self.dv=True
            self.dt = self.etime - self.stime
            self.next_state()
        elif ("sending power cycle" in line ) and not (self.curr_st==2): 
            self.reset()
        else :
            pass
                                            
        

Welcome to JupyROOT 6.18/02


In [78]:
hDT = ROOT.TH1D("htime","alive time interval",50,0,5000)
gDT = ROOT.TGraphErrors(200)
#fname = 'daqlogs/daqlog_2022_05_28_191541_summ.log'
#fname = 'daqlogs_sep/daqlog_2022_06_01-14_summ.log'
#infile = open(fname)
#line = "05/28/2022 07:16:04 PM event stored"
k = 0
#ddir="daqlogs_ok"
ddir="daqlogs_sep"
for fname in os.listdir(ddir):
    infile = open(ddir + "/" + fname)
    txt_proc = fsm()
    for line in infile:
        txt_proc.process_line(line)
        #if (txt_proc.dv and txt_proc.stime<datetime(2022,5,29,13,40)):
        if (txt_proc.dv and txt_proc.stime<datetime(2022,9,17,17,30)):
            DT = txt_proc.dt.total_seconds()
            RT = txt_proc.stime - txt_proc.time0
            RT = RT.total_seconds()
            hDT.Fill(float(DT))
            gDT.SetPoint(k,float(RT),float(DT))
            k += 1
    infile.close()


Warning in <TROOT::Append>: Replacing existing TH1: htime (Potential memory leak).


In [79]:
%jsroot on
c1 = ROOT.TCanvas("c1","c1",800,600)
gDT.Draw("a*")
c1.Draw()
c0 = ROOT.TCanvas("c0","c0",800,600)
hDT.SetFillColor(ROOT.kBlue+2)
hDT.Draw("histo")
#ff = ROOT.TF1("ff","[0]*TMath::Poisson(x/[1],[2])")
ff = ROOT.TF1("ff","[0]*TMath::Exp(-[1]*x)")
#ff.SetParNames("C","s","lambda","a0","a1")
#ff.SetParNames("C","s","lambda")
ff.SetParNames("C","lambda")
ff.SetParameters(50,1/1200.)
ff.SetParLimits(0,0,200)
ff.SetParLimits(1,0.0007,0.005)

#ff.SetParameters(500,50,50,0.1,0.01)
#ff.SetParameters(500,3,600)
#hDT.Fit(ff,"S","",200,3000)
#hDT.Draw()
c0.Draw()
c0.SaveAs("FailDT_dist_sep.gif")

#print("mean time: {0:.3f} s --> {1:.3f} min".format(ff.GetParameter(1)*ff.GetParameter(2),ff.GetParameter(1)*ff.GetParameter(2)/60.))
print("mean time: {0:.3f} s --> {1:.3f} min".format(1/ff.GetParameter(1),1/ff.GetParameter(1)/60.))

mean time: 1200.000 s --> 20.000 min


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c0
Info in <TCanvas::Print>: gif file FailDT_dist_sep.gif has been created


In [4]:
line0 = "05/28/2022 07:16:04 PM event stored"
line1 = "05/28/2022 07:24:15 PM event stored"
t0 = datetime.strptime((reduce(lambda x,y: x +" "+y,line0.split()[:3])),"%m/%d/%Y %I:%M:%S %p")
t1 = datetime.strptime((reduce(lambda x,y: x +" "+y,line1.split()[:3])),"%m/%d/%Y %I:%M:%S %p")
dt = t1 - t0
print(dt.total_seconds())
print(dt)
flist=os.listdir('daqlogs_ok')
print(flist)

491.0
0:08:11
['daqlog_2022_05_26_202100_summ.log', 'daqlog_2022_05_27_161635_summ.log', 'daqlog_2022_05_28_191541_summ.log', 'daqlog_2022_05_26_115828_summ.log']


In [80]:
1078/60.

17.966666666666665